# What is the Most Popular Star Wars Movie?

<div align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/6/6c/Star_Wars_Logo.svg" width="300"/>
</div>

<p align="center">✨ May the Force be with you! ✨</p>

---

## Table of Contents
1. Introduction
2. Data Collection
3. Data Exploration
    - Initial Observations
    - Samples
    - Unique Values
4. Data Preprocessing
    - Yes or No Columns
    - One-hot encoding columns
    - Ranking Columns

---

## 📝 Introduction

This notebook is an analysis of the Star Wars survey to find out the most popular Star Wars movie. We will explore the data, clean it, and use it to answer our main question: which Star Wars film is the fan favorite?

## 📊 Data Collection

The data comes from a survey of Star Wars fans conducted using SurveyMonkey. There were 835 responses in total, and the results were uploaded to a [GitHub repository](https://github.com/fivethirtyeight/data/tree/master/star-wars-survey).

When loading the data, we use a special encoding (ISO-8859-1) because the dataset contains some characters that aren't supported by Python's default encoding.

## 🔍 Data Exploration

### 👀 Initial Observations

Before diving into detailed analysis, we will first explore the **general information** of the dataset to understand its **structure** and identify any initial issues or patterns. This will help guide our **data cleaning** and **preparation** steps.

In [265]:
import seaborn as sns
import pandas as pd 
import numpy as np
import sys
import os

In [266]:
# load the dataset
df = pd.read_csv('starWars.csv', encoding='ISO-8859-1') # ISO-8859-1 encoding to handle special characters

In [267]:
# print the general information of the dataset
print('*' * 100)
print('The information of the dataset is as follows:')
print(df.info())

****************************************************************************************************
The information of the dataset is as follows:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187 entries, 0 to 1186
Data columns (total 38 columns):
 #   Column                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                         --------------  -----  
 0   RespondentID                                                                                                                                   1186 non-null   float64
 1   Have you seen any of the 6 films in the Star Wars franchise?                                                                                   1187 non-null   object 
 2   Do you consider yourself to be a fan of the Star Wars

---

At first glance, we notice the following about the dataset:
- There are **38 features** — we need to **normalize** them.
- Some features are **unnamed** — we should determine what these columns **represent**.
- Except for the response ID (a float), all other columns are **strings** - the format is consistent.
- Several columns contain **many null values** - the significant **data preprocessing** will be needed.

Overall, this dataset will require substantial cleaning.  
The next step of the plan is to simplify the dataset for easier analysis:
1. We can make the data a bit easier to analyze by converting 'yes or no' column to a **Boolean**.
2. Investigate unnamed columns, as some may be **one-hot encoded** or represent **rankings**.

---

### 🧾 Samples

From our initial observations, it’s clear that the dataset has a **complex structure** and will require **cleaning** and **restructuring**.

After gaining a general understanding of the dataset’s structure, we can now look at some sample entries.

In [268]:
# see some examples of the dataset
df.head()


,RespondentID,Have you seen any of the 6 films in the Star Wars franchise?,Do you consider yourself to be a fan of the Star Wars film franchise?,Which of the following Star Wars films have you seen? Please select all that apply.,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Please rank the Star Wars films in order of preference with 1 being your favorite film in the franchise and 6 being your least favorite film.,...,Unnamed: 28,Which character shot first?,Are you familiar with the Expanded Universe?,Do you consider yourself to be a fan of the Expanded Universe?æ,Do you consider yourself to be a fan of the Star Trek franchise?,Gender,Age,Household Income,Education,Location (Census Region)
0,NaN,Response,Response,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,Star Wars: Episode I The Phantom Menace,...,Yoda,Response,Response,Response,Response,Response,Response,Response,Response,Response
1,3.292880e+09,Yes,Yes,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,3,...,Very favorably,I don't understand this question,Yes,No,No,Male,18-29,NaN,High school degree,South Atlantic
2,3.292880e+09,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yes,Male,18-29,"$0 - $24,999",Bachelor degree,West South Central
3,3.292765e+09,Yes,No,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,NaN,NaN,NaN,1,...,Unfamiliar (N/A),I don't understand this question,No,NaN,No,Male,18-29,"$0 - $24,999",High school degree,West North Central
4,3.292763e+09,Yes,Yes,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,5,...,Very favorably,I don't understand this question,No,NaN,Yes,Male,18-29,"$100,000 - $149,999",Some college or Associate degree,West North Central


---

From the samples above, it is clear that this dataset requires significant cleaning. **It contains a mix of multiple-choice answers, yes-or-no responses, and one-hot encoded columns.**

---

### 🔑 Unique Values

From the sample rows, we can see that some columns are simple **yes-or-no questions**, some are **one-hot encoded** columns, and others are **multiple choice**. To simplify the dataset, we will ignore the one-hot encoded columns and **focus on exploring the unique values** in each of the remaining columns.

In [269]:
# display the unique answers of each column
print('*' * 100)

# category the columns
one_hot_encoding_columns = ["Which of the following Star Wars films have you seen? Please select all that apply.",
                           "Please rank the Star Wars films in order of preference with 1 being your favorite film in the franchise and 6 being your least favorite film.",
                           "Please state whether you view the following characters favorably, unfavorably, or are unfamiliar with him/her."]

# loop through all the columns and then print the unique vallues
for col in df.columns:
    if not col.startswith('Unnamed') and col not in one_hot_encoding_columns:
        print(f"Unique values in '{col}':")
        print(df[col].unique())
        print('*' * 100)

****************************************************************************************************
Unique values in 'RespondentID':
[           nan 3.29288000e+09 3.29287954e+09 ... 3.28837529e+09
 3.28837307e+09 3.28837292e+09]
****************************************************************************************************
Unique values in 'Have you seen any of the 6 films in the Star Wars franchise?':
['Response' 'Yes' 'No']
****************************************************************************************************
Unique values in 'Do you consider yourself to be a fan of the Star Wars film franchise?':
['Response' 'Yes' nan 'No']
****************************************************************************************************
Unique values in 'Which character shot first?':
['Response' "I don't understand this question" nan 'Greedo' 'Han']
****************************************************************************************************
Unique values in 'Are you f

---

**From the unique values, we can see that there are basically two types of columns: "yes or no" and "multiple choice".**

**Yes or no columns:**
- Have you seen any of the 6 films in the Star Wars franchise?
- Do you consider yourself to be a fan of the Star Wars film franchise?
- Are you familiar with the Expanded Universe?
- Do you consider yourself to be a fan of the Expanded Universe?
- Do you consider yourself to be a fan of the Star Trek franchise?

**Multiple choice columns:**
- Gender — the respondent's gender
- Age — the respondent's age group
- Household Income — the respondent's income bracket
- Education — the respondent's education level
- Location — the respondent's census region

---

## 🛠️ Data Preprocessing

### ✔️ Yes or No Columns

Currently, the yes-or-no columns use string values (**"Yes"** and **"No"**). To simplify analysis, we will convert these columns to boolean values: **True**, **False**. Using booleans makes it easier to **filter** and **analyze** the data without relying on string comparisons.

We will use the pandas map() method to perform this conversion.

In [270]:
# convert the yes or no answers to boolean values
conversion_map = {
    "Yes": True,
    "No": False,
}

# group the yes or no columns
conversion_columns = [
"Have you seen any of the 6 films in the Star Wars franchise?",
"Do you consider yourself to be a fan of the Star Wars film franchise?",
"Are you familiar with the Expanded Universe?",
"Do you consider yourself to be a fan of the Expanded Universe?æ",
"Do you consider yourself to be a fan of the Star Trek franchise?"]

# convert the columns to boolean values
for col in conversion_columns:
    df[col] = df[col].map(conversion_map)


In [271]:
df.head()

,RespondentID,Have you seen any of the 6 films in the Star Wars franchise?,Do you consider yourself to be a fan of the Star Wars film franchise?,Which of the following Star Wars films have you seen? Please select all that apply.,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Please rank the Star Wars films in order of preference with 1 being your favorite film in the franchise and 6 being your least favorite film.,...,Unnamed: 28,Which character shot first?,Are you familiar with the Expanded Universe?,Do you consider yourself to be a fan of the Expanded Universe?æ,Do you consider yourself to be a fan of the Star Trek franchise?,Gender,Age,Household Income,Education,Location (Census Region)
0,NaN,NaN,NaN,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,Star Wars: Episode I The Phantom Menace,...,Yoda,Response,NaN,NaN,NaN,Response,Response,Response,Response,Response
1,3.292880e+09,True,True,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,3,...,Very favorably,I don't understand this question,True,False,False,Male,18-29,NaN,High school degree,South Atlantic
2,3.292880e+09,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,True,Male,18-29,"$0 - $24,999",Bachelor degree,West South Central
3,3.292765e+09,True,False,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,NaN,NaN,NaN,1,...,Unfamiliar (N/A),I don't understand this question,False,NaN,False,Male,18-29,"$0 - $24,999",High school degree,West North Central
4,3.292763e+09,True,True,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi,5,...,Very favorably,I don't understand this question,False,NaN,True,Male,18-29,"$100,000 - $149,999",Some college or Associate degree,West North Central


---

The table above **confirms** that the original "Yes" and "No" string values have been **successfully** converted to boolean values: True and False.

---

### 🎬 One-hot encoding columns

There are several columns represent **a single checkbox question**. The respondent checked off a series of boxes in response to the question, 

For example:

- **Which of the following Star Wars films have you seen? Please select all that apply.**
— Indicates whether the respondent has seen **Star Wars: Episode I – The Phantom Menace**.
- **Unnamed: 4** — Indicates whether the respondent has seen **Star Wars: Episode II – Attack of the Clones**.
- **Unnamed: 5** — Indicates whether the respondent has seen **Star Wars: Episode III – Revenge of the Sith**.
- **Unnamed: 6** — Indicates whether the respondent has seen **Star Wars: Episode IV – A New Hope**.
- **Unnamed: 7** — Indicates whether the respondent has seen **Star Wars: Episode V – The Empire Strikes Back**.
- **Unnamed: 8** — Indicates whether the respondent has seen **Star Wars: Episode VI – Return of the Jedi**.

For each of these columns, if a cell contains the movie title, it indicates the respondent has **seen** that movie.

We should convert these columns to **boolean values** and give them more descriptive names. Similar to our previous approach, we’ll map the **movie title** to **True** and **NaN** to **False** in each column.

In [272]:
# convert the 'Unnamed' columns to boolean values
for col in df.columns[3:9]:
    df[col] = df[col].notna()

In [273]:
df.head()

,RespondentID,Have you seen any of the 6 films in the Star Wars franchise?,Do you consider yourself to be a fan of the Star Wars film franchise?,Which of the following Star Wars films have you seen? Please select all that apply.,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Please rank the Star Wars films in order of preference with 1 being your favorite film in the franchise and 6 being your least favorite film.,...,Unnamed: 28,Which character shot first?,Are you familiar with the Expanded Universe?,Do you consider yourself to be a fan of the Expanded Universe?æ,Do you consider yourself to be a fan of the Star Trek franchise?,Gender,Age,Household Income,Education,Location (Census Region)
0,NaN,NaN,NaN,True,True,True,True,True,True,Star Wars: Episode I The Phantom Menace,...,Yoda,Response,NaN,NaN,NaN,Response,Response,Response,Response,Response
1,3.292880e+09,True,True,True,True,True,True,True,True,3,...,Very favorably,I don't understand this question,True,False,False,Male,18-29,NaN,High school degree,South Atlantic
2,3.292880e+09,False,NaN,False,False,False,False,False,False,NaN,...,NaN,NaN,NaN,NaN,True,Male,18-29,"$0 - $24,999",Bachelor degree,West South Central
3,3.292765e+09,True,False,True,True,True,False,False,False,1,...,Unfamiliar (N/A),I don't understand this question,False,NaN,False,Male,18-29,"$0 - $24,999",High school degree,West North Central
4,3.292763e+09,True,True,True,True,True,True,True,True,5,...,Very favorably,I don't understand this question,False,NaN,True,Male,18-29,"$100,000 - $149,999",Some college or Associate degree,West North Central


Next, we'll need to rename the columns to reflect what they represent. We can use pandas rename() method to do this. 

In [274]:
df = df.rename(columns={
    "Which of the following Star Wars films have you seen? Please select all that apply.": "Seen_1",
    "Unnamed: 4": "Seen_2",
    "Unnamed: 5": "Seen_3",
    "Unnamed: 6": "Seen_4",
    "Unnamed: 7": "Seen_5",
    "Unnamed: 8": "Seen_6"})

In [275]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187 entries, 0 to 1186
Data columns (total 38 columns):
 #   Column                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                         --------------  -----  
 0   RespondentID                                                                                                                                   1186 non-null   float64
 1   Have you seen any of the 6 films in the Star Wars franchise?                                                                                   1186 non-null   object 
 2   Do you consider yourself to be a fan of the Star Wars film franchise?                                                                          836 non-null    object 
 3   Seen_1                      

---

After converting the one-hot encoding columns to **boolean values** and giving them **clear names**, the dataset is now much easier to work with. This step helps us quickly see which movies each respondent has seen and sets us up for more straightforward analysis in the next sections.

---

### 🏆 Ranking Columns

Next, we will try to clean the ranking columns. 

The next six columns ask the respondent to **rank** the Star War movies in order **from least to most favorite**. 1 means the film was the most favorite, and 6 means it was the least favorite. Each of the follwing columns can contain the value 1, 2, 3, 4, 5, 6, or NaN.

- **Please rank the Star Wars films in order of preference with 1 being your favorite film in the franchise and 6 being your least favorite film.** - How much the respondent liked Star Wars: Episode I The Phantom Menace
- **Unnamed: 10** - How much the respondent liked Star Wars: Episode II Attack of the Clones
- **Unnamed: 11** - How much the respondent liked Star Wars: Episode III Revenge of the Sith
- **Unnamed: 12** - How much the respondent liked Star Wars: Episode IV A New Hope
- **Unnamed: 13** - How much the respondent liked Star Wars: Episode V The Empire Strikes Back
- **Unnamed: 14** - How much the respondent liked Star Wars: Episode VI Return of the Jedi


Fortunately, these columns don't require much cleaning. We'll need to convert each column to a numeric type, then rename the columns so that we can tell what they represent more easily.

In [276]:
# rename for clarity
df = df.rename(columns={
    "Please rank the Star Wars films in order of preference with 1 being your favorite film in the franchise and 6 being your least favorite film.": "ranking_1",
    "Unnamed: 10": "ranking_2",
    "Unnamed: 11": "ranking_3",
    "Unnamed: 12": "ranking_4",
    "Unnamed: 13": "ranking_5",
    "Unnamed: 14": "ranking_6"
})

In [277]:
# Convert ranking columns to float for analysis, skipping the first row
ranking_columns = [
    "ranking_1", "ranking_2", "ranking_3", "ranking_4", "ranking_5", "ranking_6"
]
df.loc[1:, ranking_columns] = df.loc[1:, ranking_columns].astype(float)

In [278]:
df[["ranking_1", "ranking_2", "ranking_3", "ranking_4", "ranking_5", "ranking_6"]].head()

,ranking_1,ranking_2,ranking_3,ranking_4,ranking_5,ranking_6
0,Star Wars: Episode I The Phantom Menace,Star Wars: Episode II Attack of the Clones,Star Wars: Episode III Revenge of the Sith,Star Wars: Episode IV A New Hope,Star Wars: Episode V The Empire Strikes Back,Star Wars: Episode VI Return of the Jedi
1,3.0,2.0,1.0,4.0,5.0,6.0
2,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,2.0,3.0,4.0,5.0,6.0
4,5.0,6.0,1.0,2.0,4.0,3.0
